# Testing Notebook for YRBSS R code

When you run this notebook, you need to adjust file paths to fit your setup.

First, load the source code.

In [1]:
source('c:/Users/Rex/Desktop/yrbs/R/YRBSS.R')
"OK"

Warning message:
"package 'survey' was built under R version 3.3.2"Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: 'survey'

The following object is masked from 'package:graphics':

    dotchart



[1] "OK"

Now do some smoke-testing.

We load the national data and perform various operations.

First we test the function that loads the data file and extract the right state and year from the data.

In [2]:
rda.path = "C:/Users/Rex/Desktop/OWH/owh-ds/software/owh/backoffice/yrbs/rda/"
rda.path = "C:/Users/Rex/Desktop/yrbs/rda/"
load(paste0(rda.path,"sadc_2015_national.rda"))
kolumns = colnames(sadc.2015.national.df)
#cat(kolumns)
odd = c(1,3,5,7,9)
gc()

### These just check that we can load the appropriate state and year, and return
### an error message if not.
dim.l.y = function(state,year) nrow(get.df.for.state.and.year(state,year,rda.path))    
print(dim.l.y('XX', 2015))
print(dim.l.y('XX', 2011))
print(dim.l.y('XX', 2009))
print(dim.l.y('MT', 2015))
print(dim.l.y('MT', 2011))
print(dim.l.y('MT', 2009))
print(dim.l.y('NC', 2015))
print(dim.l.y('NC', 2011))
print(dim.l.y('NC', 2009))
print(get.df.for.state.and.year('MT', 1909, rda.path))
gc()
    
      

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1222691,65.3,2164898,115.7,1443371,77.1
Vcells,45038138,343.7,59250972,452.1,45069917,343.9


[1] 15624
[1] 15425
[1] 16410
[1] 4486
[1] 4148
[1] 1852
[1] 6178
[1] 2278
[1] 5702
[1] "ERROR: No data for state MT and year 1909!"


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1227224,65.6,2164898,115.7,1443371,77.1
Vcells,45043266,343.7,172430276,1315.6,215522433,1644.4


Next we smoke-test yrbssCalculate.  We focus on Q8 (Bicycle Helmet Use) and Q9 (Drinking Driver).  We just make a number of calls to be sure the function returns a matrix of the right shape and numbers of the right scale (percentages).  We have a couple of cases to check basic error handling.

In [3]:
### These are just a smoke test of yrbssCalculate.  Arguments that would normally be passed are
### allowed to default.
cat('\n\n', yrbssCalculate(filter=list(q8=1:2), rda.path=rda.path))
cat('\n\n', yrbssCalculate(filter=list(q8=1:2), state='MT', year = 2011, rda.path=rda.path) )
cat('\n\n', yrbssCalculate(filter=list(q8=1:2), state='MM', year = 2011, rda.path=rda.path) )
cat('\n\n', yrbssCalculate(filter=list(q9=c(1,3,5)), rda.path=rda.path) )
cat('\n\n', yrbssCalculate(filter=list(q8=1:2, q9=c(1,3,5)), rda.path=rda.path) )
cat('\n\n', yrbssCalculate(filter=list(q1=c(4, 5, 6), # age 15-18
                            q2=c(1), # female
                            q5=c("C","E")), rda.path=rda.path)) # bogus
cat('\n\n', yrbssCalculate(filter=list(queue8=1:2), rda.path=rda.path) )  
    
### These examples illustrate caching.  We cache a dataframe for Montana/2011
### and use it twice.  Third time we get an error.
df.MT.2011 = get.df.for.state.and.year('MT',2011,rda.path)
cat('\n\n', yrbssCalculate(filter=list(q8=1:2), state='MT', year = 2011, df=df.MT.2011) )
cat('\n\n', yrbssCalculate(filter=list(q9=odd), state='MT', year = 2011, df=df.MT.2011) )
cat('\n\n', yrbssCalculate(filter=list(q8=1:2), state='MS', year = 2013, df=df.MT.2011) )



 sex,grade,q8.count,q8.pct,q8.se,q8.ci.lo,q8.ci.hi,q9.count,q9.pct,q9.se,q9.ci.lo,q9.ci.hi
1,1,1327,58.26,2.72,52.69,63.63,1327,6.55,1.04,4.73,9.01
2,1,1281,74.06,2,69.82,77.89,1281,7.87,1.08,5.94,10.35
-1,1,2614,66.46,2.13,62.02,70.62,2614,7.25,0.82,5.75,9.1
1,2,1371,56.23,3.33,49.44,62.8,1371,4.81,0.97,3.2,7.18
2,2,1340,69.75,2.02,65.52,73.67,1340,8.55,1.44,6.05,11.95
-1,2,2725,62.72,1.91,58.79,66.49,2725,6.58,1.07,4.72,9.11
1,3,1454,48.89,3.51,41.87,55.95,1454,4.58,0.8,3.22,6.49
2,3,1427,66.36,2.43,61.28,71.08,1427,7.99,1.66,5.22,12.05
-1,3,2890,57.86,2.32,53.11,62.46,2890,6.52,1.14,4.57,9.23
1,4,1368,50.34,2.9,44.49,56.17,1368,5.32,0.96,3.69,7.63
2,4,1359,59.49,2.41,54.56,64.24,1359,6.58,1.35,4.32,9.88
-1,4,2730,54.91,2.05,50.74,59.01,2730,5.95,0.97,4.26,8.24
1,-1,5539,53.55,2.65,48.17,58.85,5539,5.39,0.7,4.14,7
2,-1,5435,67.54,1.33,64.8,70.18,5435,7.92,0.95,6.21,10.06
-1,-1,11018,60.59,1.69,57.12,63.95,11018,6.71,0.75,5.34,8.4


 sex,grade,q8.count,q8.pct,q8.se,q8.ci.lo,q8.ci.hi

Now we do a more systematic test of the group-by functionality.  We set up a contrived matrix, group by (nearly) all possible pairs of columns, and look at all remaining columns as response variables.

In [4]:
test.vec = function (i,n) c(rep(1,i), rep(0,n-i))    
m = sapply(1:20,test.vec,20)
m = rbind(m,matrix(rep(NA,100),ncol=20))
m = data.frame(m)
m$PSU = 1:25
m$stratum = rep(1,25)
m$weight = rep(1,25)
m

X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X14,X15,X16,X17,X18,X19,X20,PSU,stratum,weight
1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,2,1,1
0,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,3,1,1
0,0,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,4,1,1
0,0,0,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,5,1,1
0,0,0,0,0,1,1,1,1,1,...,1,1,1,1,1,1,1,6,1,1
0,0,0,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,7,1,1
0,0,0,0,0,0,0,1,1,1,...,1,1,1,1,1,1,1,8,1,1
0,0,0,0,0,0,0,0,1,1,...,1,1,1,1,1,1,1,9,1,1
0,0,0,0,0,0,0,0,0,1,...,1,1,1,1,1,1,1,10,1,1


In [5]:
correct.column3 = function(g1,g2,r) {
    if (r>g1 && r<g2) {
        c(20-g2, 0, 20-g2, g2-g1, g1, g2, 20-g1, g1, 20)
    }
}

correct.column3(4,17,12)

[1]  3  0  3 13  4 17 16  4 20

In [6]:
res = yrbssCalculate(df=m, state='99', group.by=list("X4","X17"), positives=list(X12=1), filter=NULL, out.format="matrix")
res

X4,X17,X12.count,X12.pct,X12.se,X12.ci.lo,X12.ci.hi
0,0,3,0.00,0.00,NaN,NaN
1,0,0,0.00,0.00,0.00,0.00
-1,0,3,0.00,0.00,NaN,NaN
0,1,13,61.54,13.77,32.50,84.17
1,1,4,100.00,0.00,NaN,NaN
-1,1,17,70.59,11.28,43.89,88.05
0,-1,16,50.00,12.76,25.86,74.14
1,-1,4,100.00,0.00,NaN,NaN
-1,-1,20,48.00,10.20,28.43,68.21


In [7]:
any(res[,3]!=correct.column3(4,17,12))

[1] FALSE

In [8]:
test.column = function(k) function(m,g1,g2,r) {
    group.by = as.list(paste0("X", c(g1,g2)))
    pos.name = paste0("X",r)
    positives = NULL
    positives[[pos.name]] = 1
    res = yrbssCalculate(df=m, state="99", group.by=group.by, positives=positives, filter=NULL, out.format="matrix")
    res[,k]
}

test.column4 = test.column(4)
test.column3 = test.column(3)

In [9]:
test.column3(m,4,17,12)

[1]  3  0  3 13  4 17 16  4 20

In [10]:
correct.column4 = function(g1,g2,r) {
    v =
    c(0,
      0,
      0,
     (r-g1)/(g2-g1),
      1,
      r/g2,
      (r-g1)/(20-g1),
      1,
      r/25)  
    return(round(100*v,2))
}
correct.column4(4,17,12)

[1]   0.00   0.00   0.00  61.54 100.00  70.59  50.00 100.00  48.00

In [11]:
test.matrix = function () {
    errors = 0
    for (g1 in 1:17) {
        for (g2 in (g1+2):19) {  # would have to rewrite correct.column3 to check g2=20
            for (r in (g1+1):(g2-1)) {
                if (any(test.column3(m,g1,g2,r) != correct.column3(g1,g2,r))) {
                    cat("Failure for (g1,g2,r) column 3 = ", c(g1,g2,r), "\n")
                    errors = errors + 1
                }  
                if (any(test.column4(m,g1,g2,r) != correct.column4(g1,g2,r))) {
                    cat("Failure for (g1,g2,r) column 4 = ", c(g1,g2,r), "\n")
                    cat(test.column4(m,g1,g2,r),"\n",correct.column4(g1,g2,r), "\n\n")
                    errors = errors + 1
                }                
            }
        }
    }
    cat(errors, " errors in m\n")
    errors = 0
    m1 = m[sample(25),]  # permute rows and try again.
    for (g1 in 1:17) {
        for (g2 in (g1+2):19) {  # would have to rewrite correct.column3 to check g2=20
            for (r in (g1+1):(g2-1)) {
                if (any(test.column3(m1,g1,g2,r) != correct.column3(g1,g2,r))) {
                    cat("Failure for (g1,g2,r) column 3 = ", c(g1,g2,r), "\n")
                    errors = errors + 1
                }   
                if (any(test.column4(m1,g1,g2,r) != correct.column4(g1,g2,r))) {
                    cat("Failure for (g1,g2,r) column 4 = ", c(g1,g2,r), "\n")
                    cat(test.column4(m,g1,g2,r),"\n",correct.column4(g1,g2,r), "\n\n")
                    errors = errors + 1
                }                
            }
        }
    }
    cat(errors, " errors in m1\n")
}

In [12]:
test.matrix()

0  errors in m
0  errors in m1


In [13]:
test.column3(m,9,20,18)  # is correct.

[1] 11  9 20 11  9 20

Now some systematic testing of filtering.  We set up a matrix in which the columns are exactly the binary representation of the integers 0 to 127.  Then we filter on combinations of columns, and look at another column as response variable.  We should always find a proportion of 1/2, by construction.

In [14]:
### Now a new matrix....
kol = function(i,m) rep(c(rep(0,2^(i-1)), rep(1,2^(i-1))), 2^(m-i))
kol(3,5)

[1] 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1

In [15]:
m = data.frame(sapply(1:6, kol, 7))
m$PSU = 1:2^7
m$weight = rep(1,2^7)
m$stratum = rep(1,2^7)
m

X1,X2,X3,X4,X5,X6,PSU,weight,stratum
0,0,0,0,0,0,1,1,1
1,0,0,0,0,0,2,1,1
0,1,0,0,0,0,3,1,1
1,1,0,0,0,0,4,1,1
0,0,1,0,0,0,5,1,1
1,0,1,0,0,0,6,1,1
0,1,1,0,0,0,7,1,1
1,1,1,0,0,0,8,1,1
0,0,0,1,0,0,9,1,1
1,0,0,1,0,0,10,1,1


In [16]:
yrbssCalculate(df=m,state="99",filter=list(X1=0,X3=1), positive=list(X2=1), group.by=NULL, out.format="matrix")

X2.count,X2.pct,X2.se,X2.ci.lo,X2.ci.hi
32,50,8.87,33.13,66.87
32,50,8.87,33.13,66.87


In [17]:
errors = 0
for (i in 1:5) {
    for (j in 1:10) {
        s = sample(6,i+1)
        fil = s[-1]
        pos = s[1]
        fil = paste0("X",fil)
        pos = paste0("X",pos)
        positive=NULL
        positive[[pos]] = sample(0:1, 1)
        filters = NULL
        for (x in fil) {
            filters[[x]] = sample(0:1, 1)
        }
        mat = yrbssCalculate(df=m,state="99",filter=filters, positive=positive, group.by=NULL, out.format="matrix")
        if (mat[1,1] != 2^(7-i) | mat[2,1] != 2^(7-i) | mat[1,2] != 50 | mat[2,2] != 50) {
            print("ERROR!")
            print(mat)
            errors = errors + 1
        }
    }
}
cat(errors, "errors.")

0 errors.

As bugs are found and fixed, regression tests should be added below.